In [1]:
import numpy as np
import pandas as pd
import nltk

In [2]:
#Download requirements
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pouri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
data = pd.read_csv('Data/digikala_comment.csv')
data

,comment
0,نسبت به قیمتش ارزش خرید داره\nجاداره، طراحیش ق...
1,چند ماهی میشه که گرفتمش‌. برای برنامه نویسی و ...
2,پراید ستون جدید
3,اقا همه چیش خوبه فقط از پایین زیاد حاشیه داره ...
4,گوسی هو اوی p10 lite سیپیو و دوربین و رمش از ا...
...,...
235,پوشش دهی صفر.اصلا پیشنهاد نمیکنم
236,نصب این فن خیلی راحته و دردسر زیادی نداره درض...
237,بی کیفیت
238,سلام ٬ چندماهی میشه این پاور بانک رو تهیه کردم...


In [4]:
# Sentence Tokenizing
from nltk.tokenize import sent_tokenize

def sentence_tokenizing(text):
    return sentence_tokenizing(text)

data['comment'] = data.apply(lambda text:sent_tokenize(text['comment']), axis=1)
data

,comment
0,[نسبت به قیمتش ارزش خرید داره\nجاداره، طراحیش ...
1,"[چند ماهی میشه که گرفتمش‌., برای برنامه نویسی ..."
2,[پراید ستون جدید]
3,[اقا همه چیش خوبه فقط از پایین زیاد حاشیه داره...
4,[گوسی هو اوی p10 lite سیپیو و دوربین و رمش از ...
...,...
235,[پوشش دهی صفر.اصلا پیشنهاد نمیکنم]
236,[نصب این فن خیلی راحته و دردسر زیادی نداره در...
237,[بی کیفیت]
238,[سلام ٬ چندماهی میشه این پاور بانک رو تهیه کرد...


In [5]:
import re
import string

def clean_fa_text(text):
    #removing english characters and signs
    text = ''.join([i for i in text if not ((65 <= ord(i) <91)
                                            or (97 <= ord(i) < 123)
                                            or (48 <= ord(i) < 58))])

    #removing nbsp
    text_list = []
    for char in text:
        if ord(char) == 160:
            text_list.append(' ')
            continue
        text_list.append(char)
    text = ''.join(text_list)

    # removing sign
    text = ''.join([i for i in text if ord(i) not in [33, 34, 35, 36, 37, 38,
                                                      39, 40, 41, 42, 43, 44,
                                                      45, 46, 47, 58, 59, 60,
                                                      61, 62, 63, 64, 91, 92,
                                                      93, 94, 95, 96, 123, 124,
                                                      125, 126, 1548, 1567]])
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

def apply_clean(list_sentence: list):
    new_list = []
    for sent in list_sentence:
        new_list.append(clean_fa_text(sent))
    return new_list

data['comment'] = data['comment'].apply(apply_clean)
data

,comment
0,[نسبت به قیمتش ارزش خرید دارهجاداره طراحیش قشن...
1,"[چند ماهی میشه که گرفتمش‌, برای برنامه نویسی و..."
2,[پراید ستون جدید]
3,[اقا همه چیش خوبه فقط از پایین زیاد حاشیه داره...
4,[گوسی هو اوی سیپیو و دوربین و رمش از این خیل...
...,...
235,[پوشش دهی صفراصلا پیشنهاد نمیکنم]
236,[نصب این فن خیلی راحته و دردسر زیادی نداره در...
237,[بی کیفیت]
238,[سلام ٬ چندماهی میشه این پاور بانک رو تهیه کرد...


In [6]:
#text normalizing
from hazm import Normalizer

def apply_normalizer(list_sentence: list):
    new_list = []
    norm = Normalizer()
    for sent in list_sentence:
        new_list.append(norm.normalize(sent))
    return new_list

data['comment'] = data['comment'].apply(apply_normalizer)
data

,comment
0,[نسبت به قیمتش ارزش خرید دارهجاداره طراحیش قشن...
1,"[چند ماهی میشه که گرفتمش, برای برنامه نویسی و ..."
2,[پراید ستون جدید]
3,[اقا همه چیش خوبه فقط از پایین زیاد حاشیه داره...
4,[گوسی هو اوی سیپیو و دوربین و رمش از این خیلی ...
...,...
235,[پوشش دهی صفراصلا پیشنهاد نمی‌کنم]
236,[نصب این فن خیلی راحته و دردسر زیادی نداره درض...
237,[بی‌کیفیت]
238,[سلام ٬ چندماهی میشه این پاور بانک رو تهیه کرد...


In [7]:
#concat all sentences
def append_all(data, column: str):
    all_sentences = []
    for sent in data[column]:
        all_sentences.extend(sent)
    return all_sentences

all_sentences = append_all(data, 'comment')
all_sentences

['نسبت به قیمتش ارزش خرید دارهجاداره طراحیش قشنگهتنها مشکلش بندهای ضعیفش هست که باعث میشه استحکام چندانی نداشنه باشه',
 'چند ماهی میشه که گرفتمش',
 'برای برنامه نویسی و کارای گرافیکی ازش استفاده می\u200cکنم',
 'واقعا از هر لحاظ بگین عالیه',
 'پراید ستون جدید',
 'اقا همه چیش خوبه فقط از پایین زیاد حاشیه داره که با روشن شدن گوشی بیشتر هم میشه',
 'و نکته دیگه اینکه به خاطر این\u200cکه اطرافش یه کوچلو خمیده هست گلس بعد یه مدتی جدا مشیه',
 'ولی در کل با این قیمت بهترین گوشی هست و همه چی داره از دوربین گرفته تا رم و سی پی یو و گرافیک و حسگر\u200cهای مختلف و خیلی چیزای دیگه',
 'گوسی هو اوی سیپیو و دوربین و رمش از این خیلی بهتره خودتون میتونین برین تمام مقایسه\u200cهای و این گوشیو ببینین',
 'چادر سبک و زیباییه دوختشم عالیه جنسشم خیلی خوبه چروک نمیشه',
 'خیلی خوبه',
 'حتما پیشنهاد می\u200cکنم',
 'من چند روزیه این خردکن رو گرفتم',
 'واقعا کارکرد خوبی دارهطراحی ظاهر خردکن هم به نسبت بقیه خردکن\u200cها خیلی زیباتره',
 'تیغه\u200cهای بسیار تیزی داره که اگه موقع شستن حواستون نباشه خیلی راحت دستتون ر

In [8]:
#splite sentence wrt words

def split_sentences(data: list):
    dat=[]
    for i in range(len(data)):
        for word in data[i].split():
            dat.append(word)
    return dat

all_words = split_sentences(all_sentences)
all_words

['نسبت',
 'به',
 'قیمتش',
 'ارزش',
 'خرید',
 'دارهجاداره',
 'طراحیش',
 'قشنگهتنها',
 'مشکلش',
 'بندهای',
 'ضعیفش',
 'هست',
 'که',
 'باعث',
 'میشه',
 'استحکام',
 'چندانی',
 'نداشنه',
 'باشه',
 'چند',
 'ماهی',
 'میشه',
 'که',
 'گرفتمش',
 'برای',
 'برنامه',
 'نویسی',
 'و',
 'کارای',
 'گرافیکی',
 'ازش',
 'استفاده',
 'می\u200cکنم',
 'واقعا',
 'از',
 'هر',
 'لحاظ',
 'بگین',
 'عالیه',
 'پراید',
 'ستون',
 'جدید',
 'اقا',
 'همه',
 'چیش',
 'خوبه',
 'فقط',
 'از',
 'پایین',
 'زیاد',
 'حاشیه',
 'داره',
 'که',
 'با',
 'روشن',
 'شدن',
 'گوشی',
 'بیشتر',
 'هم',
 'میشه',
 'و',
 'نکته',
 'دیگه',
 'اینکه',
 'به',
 'خاطر',
 'این\u200cکه',
 'اطرافش',
 'یه',
 'کوچلو',
 'خمیده',
 'هست',
 'گلس',
 'بعد',
 'یه',
 'مدتی',
 'جدا',
 'مشیه',
 'ولی',
 'در',
 'کل',
 'با',
 'این',
 'قیمت',
 'بهترین',
 'گوشی',
 'هست',
 'و',
 'همه',
 'چی',
 'داره',
 'از',
 'دوربین',
 'گرفته',
 'تا',
 'رم',
 'و',
 'سی',
 'پی',
 'یو',
 'و',
 'گرافیک',
 'و',
 'حسگر\u200cهای',
 'مختلف',
 'و',
 'خیلی',
 'چیزای',
 'دیگه',
 'گوسی',
 'هو',
 'او

In [17]:
# bigram model

def create_bigram(data):
    list_of_bigrams = []
    bigram_counts = {}
    unigram_counts = {}

    for i in range(len(data)-1):
        if i < len(data) - 1 and data[i+1]:
            list_of_bigrams.append((data[i], data[i + 1]))

            if (data[i], data[i+1]) in bigram_counts:
                bigram_counts[(data[i], data[i + 1])] += 1
            else:
                bigram_counts[(data[i], data[i + 1])] = 1

        if data[i] in unigram_counts:
            unigram_counts[data[i]] += 1
        else:
            unigram_counts[data[i]] = 1

    return list_of_bigrams, unigram_counts, bigram_counts


def bigram_probability(list_of_bigrams, unigram_counts, bigram_counts):
    prob_list = {}
    for bigram in list_of_bigrams:
        word1 = bigram[0]
        prob_list[bigram] = (bigram_counts.get(bigram))/(unigram_counts.get(word1))
    return prob_list


def get_bigram(sentence: str):
    splt = sentence.split()
    bilist = []

    for i in range(len(splt) - 1):
        bilist.append((splt[i], splt[i + 1]))

    return bilist


list_of_bigrams, unigram_counts, bigram_counts = create_bigram(all_words)
bigram_prob = bigram_probability(list_of_bigrams, unigram_counts, bigram_counts)

In [24]:
#trigram model

def create_trigram(data):
    list_of_trigrams = []
    trigram_counts = {}
    bigram_counts = {}


    for i in range(len(data)-2):
        if i < len(data) - 2 and data[i+2]:
            list_of_trigrams.append((data[i], data[i + 1], data[i + 2]))

            if (data[i], data[i + 1], data[i + 2]) in trigram_counts:
                trigram_counts[(data[i], data[i + 1], data[i + 2])] += 1
            else:
                trigram_counts[(data[i], data[i + 1], data[i + 2])] = 1

        if (data[i], data[i+1]) in bigram_counts:
            bigram_counts[(data[i], data[i + 1])] += 1
        else:
            bigram_counts[(data[i], data[i + 1])] = 1

    return list_of_trigrams, bigram_counts, trigram_counts


def trigram_probability(list_of_trigrams, bigram_counts, trigram_counts):
    prob_list = {}
    for trigram in list_of_trigrams:
        word1 = trigram[0]
        word2 = trigram[1]
        prob_list[trigram] = (trigram_counts.get(trigram))/(bigram_counts.get((word1, word2)))
    return prob_list

def get_trigram(sentence: str):
    splt = sentence.split()
    bilist = []

    for i in range(len(splt) - 2):
        bilist.append((splt[i], splt[i + 1], splt[i + 2]))

    return bilist

list_of_trigrams, bigram_counts, trigram_counts = create_trigram(all_words)
trigram_prob = trigram_probability(list_of_trigrams, bigram_counts, trigram_counts)

In [11]:
#most probable bigrams
most_prob_bigram = pd.DataFrame(bigram_prob.items(), columns=['bigram', 'probability']).sort_values(by='probability', ascending=False)[:8]
most_prob_bigram

,bigram,probability
1269,"(قطعی, ازش)",1.0
5291,"(دارمش, حرف)",1.0
1506,"(میذارم, و)",1.0
5297,"(درس, خواندن)",1.0
1503,"(سفت, نیست)",1.0
5305,"(سانتیمتر, فقط)",1.0
5311,"(یک‌نفس, عمیق)",1.0
5312,"(عمیق, بکشید)",1.0


In [12]:
#most probable bigrams
most_prob_trigram = pd.DataFrame(trigram_prob.items(), columns=['trigram', 'probability']).sort_values(by='probability', ascending=False)[:8]
most_prob_trigram

,trigram,probability
3399,"(پکیج, کوچیک, بامزه)",1.0
7312,"(اشتباه, بود, خیلی)",1.0
4514,"(تراکم, پیکسلیشم, سه)",1.0
4513,"(قبای, تراکم, پیکسلیشم)",1.0
4512,"(جای, قبای, تراکم)",1.0
4511,"(اون, جای, قبای)",1.0
4510,"(سمت, اون, جای)",1.0
4509,"(میره, سمت, اون)",1.0


In [43]:
#probability calculation of test set

from hazm import Normalizer

def get_probability(list_sentence: list, ngram_probability):
    probability = 1
    for i in range(len(list_sentence)):
        if list_sentence[i] in ngram_probability:
            probability *= ngram_probability[list_sentence[i]]
        else:
            probability *= 0.00001
    return probability


sentence1 = 'این لپتاپ سخت افزار خیلی قوی داره و از پس هرکاری به راحتی برمیاد.'
sentence2 = 'این ساعت بسیار زیبا طراحی و ساخته شده'
sentence3 = 'یک محصول باکیفیت ایرانی که حقیقتا جای حمایت داره'
sentence4 = 'بوش و ماندگاری خوب هست من خیلی دوستش دارم'


normalizer = Normalizer()

sent1_bigram = get_bigram(normalizer.normalize(sentence1))
sent1_trigram = get_trigram(normalizer.normalize(sentence1))
sent1_bigram_prob = get_probability(sent1_bigram, bigram_prob)
sent1_trigram_prob = get_probability(sent1_trigram, trigram_prob)
sent1_result = {'Bigram Probability': sent1_bigram_prob, 'Trigram Probability': sent1_trigram_prob}

sent2_bigram = get_bigram(normalizer.normalize(sentence2))
sent2_trigram = get_trigram(normalizer.normalize(sentence2))
sent2_bigram_prob = get_probability(sent2_bigram, bigram_prob)
sent2_trigram_prob = get_probability(sent2_trigram, trigram_prob)
sent2_result = {'Bigram Probability': sent2_bigram_prob, 'Trigram Probability': sent2_trigram_prob}

sent3_bigram = get_bigram(normalizer.normalize(sentence3))
sent3_trigram = get_trigram(normalizer.normalize(sentence3))
sent3_bigram_prob = get_probability(sent3_bigram, bigram_prob)
sent3_trigram_prob = get_probability(sent3_trigram, trigram_prob)
sent3_result = {'Bigram Probability': sent3_bigram_prob, 'Trigram Probability': sent3_trigram_prob}

sent4_bigram = get_bigram(normalizer.normalize(sentence4))
sent4_trigram = get_trigram(normalizer.normalize(sentence4))
sent4_bigram_prob = get_probability(sent4_bigram, bigram_prob)
sent4_trigram_prob = get_probability(sent4_trigram, trigram_prob)
sent4_result = {'Bigram Probability': sent4_bigram_prob, 'Trigram Probability': sent4_trigram_prob}

result_table = pd.DataFrame({'Bigram Probability': [sent1_bigram_prob, sent2_bigram_prob, sent3_bigram_prob, sent4_trigram_prob],
                             'Trigram Probability': [sent1_trigram_prob, sent2_trigram_prob, sent3_trigram_prob, sent4_trigram_prob]},
                            index=['sentence1', 'sentence2', 'sentence3', 'sentence4'])
result_table

,Bigram Probability,Trigram Probability
sentence1,3.348454e-26,1.000000e-35
sentence2,1.288525e-08,1.250000e-01
sentence3,1.468860e-16,1.000000e-15
sentence4,3.703704e-02,3.703704e-02
